In [ ]:
!pip install pypdf

In [1]:
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

warnings.filterwarnings("ignore")

In [2]:
data_folder = p.cwd() / "data"
p(data_folder).mkdir(parents=True, exist_ok=True)

In [3]:
#Reading 'Customer-Info-Sheet-SuperSaver-Account-1.pdf' from bankofireland.com website

pdf_url = "https://personalbanking.bankofireland.com/app/uploads/Customer-Info-Sheet-SuperSaver-Account-1.pdf"
pdf_file = str(p(data_folder, pdf_url.split("/")[-1]))

urllib.request.urlretrieve(pdf_url, pdf_file)

('/home/jupyter/data/Customer-Info-Sheet-SuperSaver-Account-1.pdf',
 <http.client.HTTPMessage at 0x7f719577ad40>)

In [4]:
pdf_loader = PyPDFLoader(pdf_file)
pages = pdf_loader.load_and_split()

In [5]:
#document content

pages[0].page_content

'Minimum monthly amount €5\nMaximum monthly amount €2,500\nAccess You have instant access to your money.\nAdditional lodgements No\nWithdrawal arrangementsYou can withdraw money in any Bank of Ireland branch (you will need to bring \nphoto ID with you), by 365 online or via phone banking when registered for \nthese services.\nInterest rate type Fixed – The interest rate stays the same for the 12 months.\nInterest paymentInterest is paid at the end of the 12 month fixed rate period. See our Customer \nDeposits Rate Sheet on bankofireland.com  for current rates.\nSole or joint accounts Sole and joint accounts permitted.\nHow to applyYou can open this account online through the Bank of Ireland website via the \nmobile app or in any Bank of Ireland branch.\nTo open the account online, visit the savings page on bankofireland.com  and \ngo to the SuperSaver product page. Click the button “Apply now”.\nOther important informationTo open an account, you must have a personal current account and

In [9]:
len_pages=len(pages)

In [46]:
context = " ".join(str(p.page_content) for p in pages[:len_pages])
print("The total words in the context: ", len(context))

The total words in the context:  1733


In [10]:
def generate_answer(question):
    """
    Generates an answer to a given question using context extracted from a set of documents and Vertex AI models.

    Parameter:
    - question (str): The question for which an answer is sought.

    Steps:
    1. Initializes Vertex AI models for text generation and embeddings:
       - 'text-bison@001' for Large Language Model (LLM) text generation.
       - 'textembedding-gecko@001' for embeddings.
    2. Concatenates the contents of the first pages of previously loaded documents to form the context.
    3. Constructs a prompt template for the question-answering task using the concatenated context.
    4. Creates a QA chain using the 'text-bison@001' model with the specified prompt.
    5. Generates an answer using the QA chain based on the question and the context.

    Returns:
    str: The generated answer, or "answer not available in context" if the answer cannot be derived from the provided context.

    """
    
    vertex_llm_text = VertexAI(model_name="text-bison@001")
    vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")
    
    context = " ".join(str(p.page_content) for p in pages[:len_pages])
    
    prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                        not contained in the context, say "answer not available in context" \n\n
                        Context: \n {context}?\n
                        Question: \n {question} \n
                        Answer:
                      """

    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )
    stuff_chain = load_qa_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)
    stuff_answer = stuff_chain(
    {"input_documents": pages, "question": question}, return_only_outputs=True
    )
    return stuff_answer["output_text"]

In [22]:
question='What is the name of the savings account?'

In [23]:
generate_answer(question)

'SuperSaver Account'

In [24]:
question='Can I lodge 3000 EUR per month?'

In [25]:
generate_answer(question)

'No, the maximum monthly amount is 2,500 EUR.'

In [26]:
question='How do I withdraw the money?'

In [27]:
generate_answer(question)

'You can withdraw money in any Bank of Ireland branch (you will need to bring \nphoto ID with you), by 365 online or via phone banking when registered for \nthese services.'

In [28]:
question='What if I do not have a photo ID?'

In [29]:
generate_answer(question)

'answer not available in context'

In [30]:
question='Is there a website for this?'

In [31]:
generate_answer(question)

'Yes, the website is bankofireland.com'

In [32]:
question='Do I get variable rate?'

In [33]:
generate_answer(question)

'No, the interest rate is fixed for 12 months.'

In [34]:
question='What if I am not a customer of BOI?'

In [35]:
generate_answer(question)

'To open an account, you must have a personal current account and be aged \n16 or over.'

In [36]:
question='I have a negative balance, is this for me?'

In [37]:
generate_answer(question)

'No, you must have a positive balance to open this account.'

In [38]:
question='What if I want to buy something very expensive?'

In [39]:
generate_answer(question)

'You can withdraw money in any Bank of Ireland branch (you will need to bring \nphoto ID with you), by 365 online or via phone banking when registered for \nthese services.'

In [40]:
question='I am not convinced. Can you sell it to me?'

In [41]:
generate_answer(question)

'This account offers a fixed rate for 12 months, along with a range of extra benefits.'

In [42]:
question='I am broke, is there a chance for me?'

In [43]:
generate_answer(question)

'The minimum monthly amount is €5.'

In [44]:
question='I am 67. Is it for me?'

In [45]:
generate_answer(question) #knows where to look but did not get it right :)

'No, you must be aged 16 or over.'

In [46]:
question='Is there something in this offer that will amaze me?'

In [47]:
generate_answer(question)#????

'No, there is nothing in this offer that will amaze you.'

In [48]:
question='Is there something in this offer that I will find good for me though?'

In [49]:
generate_answer(question)

'You have instant access to your money.'

In [50]:
question='What is the interest rate?'

In [51]:
generate_answer(question)

'See our Customer Deposits Rate Sheet on bankofireland.com for current rates.'

In [52]:
question='How quickly can I save 10000?'

In [53]:
generate_answer(question)

'answer not available in context'

In [54]:
question='What about my husband?'

In [55]:
generate_answer(question)

'Yes, you can open a joint account with your husband.'

In [56]:
question='What about my children?'

In [57]:
generate_answer(question)

'Sole and joint accounts permitted.'

In [58]:
question='W is te intrs pai?' #question When is the interest paid? - spelling altered on purpose

In [59]:
generate_answer(question)

'Interest is paid at the end of the 12 month fixed rate period.'

In [60]:
question='Shall I get it?'

In [61]:
generate_answer(question) # model made a nice summary by re-formulating what is written in pdf

'The SuperSaver account is a good option for those who want to save regularly and get a fixed rate of interest for 12 months. The minimum monthly amount is €5 and the maximum monthly amount is €2,500. You can withdraw money in any Bank of Ireland branch (you will need to bring photo ID with you), by 365 online or via phone banking when registered for these services. Interest is paid at the end of the 12 month fixed rate period.'